In [1]:
pwd

'F:\\Hustle\\weather_prediction_project'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle  # For saving and loading models

# Load the dataset
data = pd.read_csv('Simplified_Weather_Data_with_City.csv')

# Convert 'datetime' to datetime object and extract year, month, and day
data['datetime'] = pd.to_datetime(data['datetime'])
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day

# One-hot encode the city names
city_encoder = OneHotEncoder()
encoded_city = city_encoder.fit_transform(data[['name']])

# Label encode the weather conditions
label_encoder = LabelEncoder()
data['conditions_encoded'] = label_encoder.fit_transform(data['conditions'])

# Prepare the feature matrix and target vectors
X = pd.concat([data[['year', 'month', 'day']], pd.DataFrame(encoded_city.toarray())], axis=1)
y_temp = data['temp']
y_humidity = data['humidity']
y_conditions = data['conditions_encoded']


In [6]:
# Split the dataset
X_train, X_test, y_train_temp, y_test_temp = train_test_split(X, y_temp, test_size=0.2, random_state=42)
X_train, X_test, y_train_humidity, y_test_humidity = train_test_split(X, y_humidity, test_size=0.2, random_state=42)
X_train, X_test, y_train_conditions, y_test_conditions = train_test_split(X, y_conditions, test_size=0.2, random_state=42)

# Initialize the models
model_temp = RandomForestRegressor(n_estimators=100, random_state=42)
model_humidity = RandomForestRegressor(n_estimators=100, random_state=42)
model_conditions = RandomForestClassifier(n_estimators=100, random_state=42)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

# Train the models
model_temp.fit(X_train, y_train_temp)
model_humidity.fit(X_train, y_train_humidity)
model_conditions.fit(X_train, y_train_conditions)

# Save the models and encoders
with open('model_temp.sav', 'wb') as file:
    pickle.dump(model_temp, file)
with open('model_humidity.sav', 'wb') as file:
    pickle.dump(model_humidity, file)
with open('model_conditions.sav', 'wb') as file:
    pickle.dump(model_conditions, file)
with open('city_encoder.sav', 'wb') as file:
    pickle.dump(city_encoder, file)
with open('label_encoder.sav', 'wb') as file:
    pickle.dump(label_encoder, file)


In [ ]:
def load_model(filename):
    with open(filename, 'rb') as file:
        return pickle.load(file)

In [28]:
def predict_weather(city, year, month, day):
    # Load models and encoders
    model_temp = load_model('model_temp.sav')
    model_humidity = load_model('model_humidity.sav')
    model_conditions = load_model('model_conditions.sav')
    city_encoder = load_model('city_encoder.sav')
    label_encoder = load_model('label_encoder.sav')

    # Encode city
    city_encoded = city_encoder.transform([[city]]).toarray()

    # Manually create city feature names based on the number of unique cities
    num_cities = city_encoded.shape[1]
    city_feature_names = [f'{i}' for i in range(num_cities)]

    # Create the features DataFrame for prediction
    feature_values = [year, month, day] + city_encoded[0].tolist()
    feature_names = ['year', 'month', 'day'] + city_feature_names
    features = pd.DataFrame([feature_values], columns=feature_names)

    # Make predictions
    predicted_temp = model_temp.predict(features)[0]
    predicted_humidity = model_humidity.predict(features)[0]
    predicted_condition_encoded = model_conditions.predict(features)[0]
    predicted_condition = label_encoder.inverse_transform([predicted_condition_encoded])[0]

    return predicted_temp, predicted_humidity, predicted_condition

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


(25.311000000000003, 75.168, 'Cloudy')

In [ ]:
# Example Prediction
predict_weather('bangalore', 2025, 7, 1)



def main():
    print("Select the City\n")
    print("1 - bangalore\n 2 - Ballari\n 3 - Hubli\n 4 - Delhi\n 5 - Mysore")
    city = input()
    

In [29]:
!python app.py

^C
